In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder.getOrCreate()


In [5]:
df = spark.read.csv('C:\\Users\\admin\\Desktop\\spark-data\\Fraud_Data.csv',inferSchema=True,header=True)

In [6]:
df.columns

['user_id',
 'signup_time',
 'purchase_time',
 'purchase_value',
 'device_id',
 'source',
 'browser',
 'sex',
 'age',
 'ip_address',
 'is_fraud']

In [7]:
df.columns[:2]

['user_id', 'signup_time']

In [10]:
F.col('source')

Column<b'source'>

In [11]:
F.expr("source")

Column<b'source'>

In [12]:
df.select('source','browser')

DataFrame[source: string, browser: string]

In [13]:
df.select('source','browser').explain()

== Physical Plan ==
*(1) FileScan csv [source#15,browser#16] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/admin/Desktop/spark-data/Fraud_Data.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<source:string,browser:string>


In [14]:
df.select('source','browser').show(5)

+------+-------+
|source|browser|
+------+-------+
|   SEO| Chrome|
|   Ads| Chrome|
|   SEO|  Opera|
|   SEO| Safari|
|   Ads| Safari|
+------+-------+
only showing top 5 rows



In [15]:
df.select(F.col('source').alias('Source'),F.col('browser').alias('Browser')).show(5)

+------+-------+
|Source|Browser|
+------+-------+
|   SEO| Chrome|
|   Ads| Chrome|
|   SEO|  Opera|
|   SEO| Safari|
|   Ads| Safari|
+------+-------+
only showing top 5 rows



In [16]:
df

DataFrame[user_id: int, signup_time: timestamp, purchase_time: timestamp, purchase_value: int, device_id: string, source: string, browser: string, sex: string, age: int, ip_address: double, is_fraud: int]

In [21]:
df.select('source','browser').withColumn('Source',F.upper(F.col('source'))).show(5)

+------+-------+
|Source|browser|
+------+-------+
|   SEO| Chrome|
|   ADS| Chrome|
|   SEO|  Opera|
|   SEO| Safari|
|   ADS| Safari|
+------+-------+
only showing top 5 rows



In [22]:
from pyspark.sql import Window

In [31]:
l = [
    (1,'sales',2700),
    (2,'admin',3700),
    (3,'sales',4700),
    (4,'admin',1500),
    (5,'dev',3000),
    (6,'dev',2700),
    (7,'sales',3200),
    (8,'dev',1900),
    (9,'dev',4500),
    (10,'dev',4000)
]



data = spark.createDataFrame(l,schema=['id','dept','salary'])

data.show()

+---+-----+------+
| id| dept|salary|
+---+-----+------+
|  1|sales|  2700|
|  2|admin|  3700|
|  3|sales|  4700|
|  4|admin|  1500|
|  5|  dev|  3000|
|  6|  dev|  2700|
|  7|sales|  3200|
|  8|  dev|  1900|
|  9|  dev|  4500|
| 10|  dev|  4000|
+---+-----+------+



In [32]:
df = data.groupBy('dept').agg(
F.expr('collect_list(salary)').alias('list_salary'),
F.expr('avg(salary)').alias('average_salary'),
F.expr('sum(salary)').alias('summation_salary')
)

df.show()

+-----+--------------------+------------------+----------------+
| dept|         list_salary|    average_salary|summation_salary|
+-----+--------------------+------------------+----------------+
|  dev|[3000, 2700, 1900...|            3220.0|           16100|
|sales|  [2700, 4700, 3200]|3533.3333333333335|           10600|
|admin|        [3700, 1500]|            2600.0|            5200|
+-----+--------------------+------------------+----------------+



In [33]:
# windowspec = Window.partitionBy('dept')
windowspec = Window.partitionBy('dept').orderBy(F.asc('salary'))


df = data.withColumn('list_salary',F.collect_list(F.col('salary')).over(windowspec))\
.withColumn('average_salary',F.avg(F.col('salary')).over(windowspec))\
.withColumn('total_salary',F.sum(F.col('salary')).over(windowspec))


df.show()

+---+-----+------+--------------------+------------------+------------+
| id| dept|salary|         list_salary|    average_salary|total_salary|
+---+-----+------+--------------------+------------------+------------+
|  8|  dev|  1900|              [1900]|            1900.0|        1900|
|  6|  dev|  2700|        [1900, 2700]|            2300.0|        4600|
|  5|  dev|  3000|  [1900, 2700, 3000]|2533.3333333333335|        7600|
| 10|  dev|  4000|[1900, 2700, 3000...|            2900.0|       11600|
|  9|  dev|  4500|[1900, 2700, 3000...|            3220.0|       16100|
|  1|sales|  2700|              [2700]|            2700.0|        2700|
|  7|sales|  3200|        [2700, 3200]|            2950.0|        5900|
|  3|sales|  4700|  [2700, 3200, 4700]|3533.3333333333335|       10600|
|  4|admin|  1500|              [1500]|            1500.0|        1500|
|  2|admin|  3700|        [1500, 3700]|            2600.0|        5200|
+---+-----+------+--------------------+------------------+------